In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

In [ ]:
# Restrict the amount of GPU allocated to tensorflow
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [ ]:
# Load in training data (2000 img) from pickle files (too large to save through git)
# Pickle files from cnn_datacleaning.ipynb
import pickle

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

# Standardize pixel values (0-1 instead of 0-255)
X = X/255.0

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)

In [ ]:
# Model architecture
model = Sequential()

# Convolution layer 1
model.add(Conv2D(50, (3, 3), strides=(1,1), padding='same', activation='relu', input_shape=X.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Convolution layer 2
model.add(Conv2D(50, (3, 3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Convolution layer 3
model.add(Conv2D(50, (3, 3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flatten to 1D after 2D conv layers
model.add(Flatten()) 

# Hidden layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))

# Output layer
model.add(Dense(1, activation ='sigmoid'))

In [ ]:
# Compile model with AUC as the metric to train on
model.compile(
    loss='binary_crossentropy', 
    metrics=[tf.keras.metrics.AUC(num_thresholds=50)], 
    optimizer='adam')

In [ ]:
# Save checkpoint models throughout training, when validation AUC is the highest ever seen

checkpoint_filepath = '/tmp/checkpoint/melanoma_{epoch:02d}-{val_loss:.2f}'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_auc',
    mode='max',
    save_best_only=True)

In [ ]:
# Weight classes based on amount of each class label in training data
_, counts = np.unique(y, return_counts=True)

class_weights = {
    0: counts[0] / np.max(counts),
    1: 1 / (counts[1] / np.max(counts))
}


In [ ]:
# training the model for 10 epochs on training data with the X_test/y_test 
# reserved for validation between epochs
H = model.fit(
    X_train, 
    y_train, 
    batch_size=80, 
    epochs=10,  
    class_weight=class_weights, 
    validation_data=(X_test, y_test),
    verbose=1)

In [ ]:
# Save model to file
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%d_%m_%Y-%H_%M_%S")
model.save(f'melanoma_model_training_end_{current_time}')